# Transonic Airfoil Aerodynamic Coefficients Prediction using Artificial Neural Network

##Fill in your answer in "isi jawaban anda!"

In this notebook, an artificial neural network being modeled to predict the aerodynamic coefficients of an airfoil that flies at a nominal condition of __Mach Number M = 0.729__ where the data of varying M and __angle of attack α__ results in various __lift coefficient Cl__, __drag coefficient Cd__ and the pressure coefficient at the center of the upper surface of the airfoil __Cp__ using _Tensorflow and Keras_.

The data were taken from the following reference: Pramudita Satria Palar, Lavi Rizki Zuhal, and Koji Shimoyama."Gaussian Process Surrogate Model with Composite Kernel Learning for Engineering Design." _AIAA Journal_ 58, no. 4 (2020):1864-1880.

_Notebook made to explore simple Artificial Neural Network, Rafael S_

### Importing required packages

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: No module named 'tensorflow'

### Importing Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
df = pd.read_csv('transonic_airfoil_data_set.csv').drop(columns = 'No')
df

### Data Plot

Plotting the data to have a better visualization of our data distribution and linearity.

#### Lift Coefficient Cl Plot

In [ ]:
figcl = plt.figure(figsize = [6,6])
clplot = Axes3D(figcl)
clplot.scatter(df['M'], df['AoA'], df['Cl'])
clplot.set_xlabel('Mach Number')
clplot.set_ylabel('Angle of Attack (degree)')
clplot.set_zlabel('Cl')
plt.show()

plt.scatter(df['M'], df['Cl'], alpha = 0.3)
plt.xlabel('Mach Number')
plt.ylabel('Cl')
plt.title('Cl vs Mach Number')
plt.show()

plt.scatter(df['AoA'], df['Cl'], alpha = 0.3)
plt.xlabel('Angle of Attack (degree)')
plt.ylabel('Cl')
plt.title('Cl vs AoA')
plt.show()

#### Drag Coefficient Cd Plot

In [ ]:
figcd = plt.figure(figsize = [6,6])
cdplot = Axes3D(figcd)
cdplot.scatter(df['M'], df['AoA'], df['Cd'], c = 'r', alpha = 0.3)
cdplot.set_xlabel('Mach Number')
cdplot.set_ylabel('Angle of Attack (degree)')
cdplot.set_zlabel('Cd')
plt.show()

plt.scatter(df['M'], df['Cd'], alpha = 0.2, c = 'r')
plt.xlabel('Mach Number')
plt.ylabel('Cd')
plt.title('Cd vs Mach Number')
plt.show()

plt.scatter(df['AoA'], df['Cd'], alpha = 0.2, c = 'r')
plt.xlabel('Angle of Attack (degree)')
plt.ylabel('Cd')
plt.title('Cd vs AoA')
plt.show()

#### Pressure Coefficient Cp Plot

In [ ]:
figcp = plt.figure(figsize = [6,6])
cpplot = Axes3D(figcp)
cpplot.scatter(df['M'], df['AoA'], df['Cp'], c = 'g')
cpplot.set_xlabel('Mach Number')
cpplot.set_ylabel('Angle of Attack (degree)')
cpplot.set_zlabel('Cp')
plt.show()

plt.scatter(df['M'], df['Cp'], alpha = 0.3, c = 'g')
plt.xlabel('Mach Number')
plt.ylabel('Cp')
plt.title('Cp vs Mach Number')
plt.show()

plt.scatter(df['AoA'], df['Cp'], alpha = 0.3, c = 'g')
plt.xlabel('Angle of Attack (degree)')
plt.ylabel('Cp')
plt.title('Cp vs AoA')
plt.show()

### Correlation Coefficient

Checking the Pearson's correlation coefficient to measure the statistical relationship, or association, between the variables.

In [ ]:
df_heatmap = sns.heatmap(corr, annot = True, cmap = 'coolwarm')

### Input and Output Data
Defining the input data and the output data.

In [ ]:
X = df.loc[:, ["M", "AoA"]]
y = df.loc[:, ["Cl","Cd","Cp"]]

In [ ]:
X #Input Data

In [ ]:
y #Output Data

### Dataset Split (Training and Validation Data)
Splitting the dataset by random and evenly distributed for the training and validation dataset.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 29)

Plotting to visualize the dataset split result.

In [ ]:
# Creating figure
fig = plt.figure(figsize = (8, 6))
visual_split = plt.axes(projection ="3d")
 
# Creating plot
visual_split.scatter3D(X_train['M'], X_train['AoA'], y_train['Cl'], color = "green", alpha=0.3, label='Train Data')
visual_split.scatter3D(X_test['M'], X_test['AoA'], y_test['Cl'], color = "red",alpha=0.5, label='Test Data')
plt.title("Train and Test Dataset")
plt.xlabel('Mach Number')
plt.ylabel('Angle of Attack')
plt.legend()
plt.show()


## Normalizing The Data

Normalization ensures that the magnitude of the values that a feature assumes are more or less the same.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scalery = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scalery.fit_transform(y_train)
y_test = scalery.transform(y_test)

## Modeling The Artificial Neural Network

The activation function for the hidden layer of the network is using the Rectified Linear Unit (ReLU) function as it is a simple, fast and observed to converge much more quickly and reliably.

The Dropout algorithm is commented out as it doesn't provide improvements after a few trainings, and early stopping method is being used instead to avoid overfitting our neural network to the dataset.

The optimizer is the adam optimizer as it provides an adaptive learning rate to optimize better.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
NNmodel = Sequential()

NNmodel.add(Dense("relu",activation='relu')) #Hidden Layer
#NNmodel.add(Dropout(0.2))
NNmodel.add(Dense("relu",activation='relu')) #Hidden Layer
#NNmodel.add(Dropout(0.2))
NNmodel.add(Dense("relu",activation='relu')) #Hidden Layer
#NNmodel.add(Dropout(0.2))

NNmodel.add(Dense(units = 3,activation='linear')) #Output Layer

NNmodel.compile(loss='mse', optimizer='adam')

### Early Stopping

To avoid overfitting during the training of the neural network, early stopping method is being added. Early stopping is a form of regularization used to avoid overfitting when training a learner with an iterative method.

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

### Training The Artificial Neural Network

In [ ]:
NNmodel.fit(x=X_train, y=y_train, epochs=1000, validation_data=(X_test,y_test), callbacks=[early_stop])

### Plotting The Losses During Training

In [ ]:
losses = pd.DataFrame(NNmodel.history.history)
plt.grid()
plt.yscale("log")
plt.plot(losses['loss'], label='train loss')
plt.plot(losses['val_loss'], label='validation loss')
plt.legend()
losses[['loss', 'val_loss']].plot()

### Predicting On The Test Dataset

In [ ]:
predictions = NNmodel.predict(X_test)

In [ ]:
predictions = scalery.inverse_transform(predictions)
y_test = scalery.inverse_transform(y_test)

### Mean Squared Error and Mean Absolute Error of the Prediction Results

In [ ]:
mse_cl = mean_squared_error(y_test[:,0], predictions[:,0])
print(f"Mean Squared Error for Cl: {mse_cl}")
mae_cl = np.sum(np.absolute(y_test[:,0]- predictions[:,0]))/len(y_test)
print(f"Mean Absolute Error for Cl: {mae_cl}\n")
mse_cd = mean_squared_error(y_test[:,1], predictions[:,1])
print(f"Mean Squared Error for Cd: {mse_cd}")
mae_cd = np.sum(np.absolute(y_test[:,1]- predictions[:,1]))/len(y_test)
print(f"Mean Absolute Error for Cd: {mae_cd}\n")
mse_cp = mean_squared_error(y_test[:,2], predictions[:,2])
print(f"Mean Squared Error for Cp: {mse_cp}")
mae_cp = np.sum(np.absolute(y_test[:,2]- predictions[:,2]))/len(y_test)
print(f"Mean Absolute Error for Cp: {mae_cp}\n")

### Predictions vs Ground Truth Plots

In [ ]:
###############################Cl
# Creating figure Cl
fig = plt.figure(figsize = (8, 6))
visual_split = plt.axes(projection ="3d")
 
# Creating plot
visual_split.scatter3D(scaler.inverse_transform(X_test)[:,0], scaler.inverse_transform(X_test)[:,1], predictions[:,0], color = "black", alpha=1, label='Predictions', marker='x')
visual_split.scatter3D(scaler.inverse_transform(X_test)[:,0], scaler.inverse_transform(X_test)[:,1], y_test[:,0], color = "blue", alpha=0.6, label='Ground Truth')
plt.title("Cl Prediction vs Ground Truth")
plt.xlabel('Mach Number')
plt.ylabel('Angle of Attack')
plt.legend()
plt.show()

plt.scatter(scaler.inverse_transform(X_test)[:,0], predictions[:,0], color = "black", alpha=1, label='Predictions', marker='x')
plt.scatter(scaler.inverse_transform(X_test)[:,0], y_test[:,0], color = "blue", alpha=0.6, label='Ground Truth')
plt.xlabel('Mach Number')
plt.ylabel('Cl')
plt.title('Cl vs Mach Number')
plt.legend()
plt.show()

plt.scatter(scaler.inverse_transform(X_test)[:,1], predictions[:,0], color = "black", alpha=1, label='Predictions', marker='x')
plt.scatter(scaler.inverse_transform(X_test)[:,1], y_test[:,0], color = "blue", alpha=0.6, label='Ground Truth')
plt.xlabel('AoA')
plt.ylabel('Cl')
plt.title('Cl vs AoA')
plt.legend()
plt.show()

##############################Cd
# Creating figure Cd
fig = plt.figure(figsize = (8, 6))
visual_split = plt.axes(projection ="3d")
 
# Creating plot
visual_split.scatter3D(scaler.inverse_transform(X_test)[:,0], scaler.inverse_transform(X_test)[:,1], predictions[:,1], color = "black", alpha=1, label='Predictions', marker='x')
visual_split.scatter3D(scaler.inverse_transform(X_test)[:,0], scaler.inverse_transform(X_test)[:,1], y_test[:,1], color = "red", alpha=0.6, label='Ground Truth')
plt.title("Cd Prediction vs Ground Truth")
plt.xlabel('Mach Number')
plt.ylabel('Angle of Attack')
plt.legend()
plt.show()

plt.scatter(scaler.inverse_transform(X_test)[:,0], predictions[:,1], color = "black", alpha=1, label='Predictions', marker='x')
plt.scatter(scaler.inverse_transform(X_test)[:,0], y_test[:,1], color = "red", alpha=0.6, label='Ground Truth')
plt.xlabel('Mach Number')
plt.ylabel('Cd')
plt.title('Cd vs Mach Number')
plt.legend()
plt.show()

plt.scatter(scaler.inverse_transform(X_test)[:,1], predictions[:,1], color = "black", alpha=1, label='Predictions', marker='x')
plt.scatter(scaler.inverse_transform(X_test)[:,1], y_test[:,1], color = "red", alpha=0.6, label='Ground Truth')
plt.xlabel('AoA')
plt.ylabel('Cd')
plt.title('Cd vs AoA')
plt.legend()
plt.show()

##############################Cp
# Creating figure Cp
fig = plt.figure(figsize = (8, 6))
visual_split = plt.axes(projection ="3d")
 
# Creating plot
visual_split.scatter3D(scaler.inverse_transform(X_test)[:,0], scaler.inverse_transform(X_test)[:,1], predictions[:,2], color = "black", alpha=1, label='Predictions', marker='x')
visual_split.scatter3D(scaler.inverse_transform(X_test)[:,0], scaler.inverse_transform(X_test)[:,1], y_test[:,2], color = "g", alpha=0.6, label='Ground Truth')
plt.title("Cp Prediction vs Ground Truth")
plt.xlabel('Mach Number')
plt.ylabel('Angle of Attack')
plt.legend()
plt.show()

plt.scatter(scaler.inverse_transform(X_test)[:,0], predictions[:,2], color = "black", alpha=1, label='Predictions', marker='x')
plt.scatter(scaler.inverse_transform(X_test)[:,0], y_test[:,2], color = "green", alpha=0.6, label='Ground Truth')
plt.xlabel('Mach Number')
plt.ylabel('Cd')
plt.title('Cd vs Mach Number')
plt.legend()
plt.show()

plt.scatter(scaler.inverse_transform(X_test)[:,1], predictions[:,2], color = "black", alpha=1, label='Predictions', marker='x')
plt.scatter(scaler.inverse_transform(X_test)[:,1], y_test[:,2], color = "green", alpha=0.6, label='Ground Truth')
plt.xlabel('AoA')
plt.ylabel('Cd')
plt.title('Cd vs AoA')
plt.legend()
plt.show()

### Sampling Predictions

In [ ]:
sample_number = 6
#alldata=len(predictions)
print("[Cl Cd Cp]")
for i in range (sample_number):
    print(f"Sample Data {i+1}: {y_test[i]}")
    print(f"Prediction {i+1}: {predictions[i]}")
    error = np.abs(y_test[i]-predictions[i])
    print(f"error: {error}\n--------------------------------------------------")
